### Hive assignment. Task2

Compare most popular tags in year 2016 with tags in 2009. Tag popularity is the number of questions (post_type_id=1) with this tag. Output top 10 tags in format:

```tag <tab> rank in 2016 <tab> rank in 2009 <tag> tag popularity in 2016 <tag> tag popularity in 2009```

Example:

```php 5 3 1234 5678```

For 'rank' in each year use rank() function. Order by 'rank in 2016'.

In [1]:
%%writefile query.hql

USE stackoverflow_;


Overwriting query.hql


In [2]:
%%writefile -a query.hql

-- Create posts_sample_external table

DROP TABLE IF EXISTS `posts_sample_external`;
CREATE EXTERNAL TABLE IF NOT EXISTS `posts_sample_external`(
  `id` int,
  `post_type_id` tinyint,
  `date` string,
  `owner_user_id` int,
  `parent_id` int,
  `score` int,
  `favorite_count` int,
  `tags` string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
  "input.regex" = '^<row.*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\")(?=.*\\bCreationDate=\"([^"]*)\")(?=.*\\bOwnerUserId=\"(\\d+)\")?(?=.*\\bParentId=\"(\\d+)\")?(?=.*\\bScore=\"(-?\\d+)\")(?=.*\\bFavoriteCount=\"(\\d+)\")?(?=.*\\bTags=\"([^"]*)\")?.*',
  "input.regex.case.insensitive" = 'true'
)
STORED AS TEXTFILE
LOCATION
  '/data/stackexchange100/posts'
;


Appending to query.hql


In [3]:
%%writefile -a query.hql

-- Create managed table and fill data

DROP TABLE IF EXISTS `posts_sample`;
CREATE TABLE IF NOT EXISTS `posts_sample`(
  `id` int,
  `post_type_id` tinyint,
  `date` string,
  `owner_user_id` int,
  `parent_id` int,
  `score` int,
  `favorite_count` int,
  `tags` array <string>
)
PARTITIONED BY ( 
  `year` string, 
  `month` string
)
-- CLUSTERED BY ( 
--   `date`
-- ) 
-- SORTED BY ( 
--   id ASC
-- ) 
-- INTO 8 BUCKETS
STORED AS TEXTFILE
LOCATION
--  '/user/hjudge/task1'
  '/user/jovyan/task1'
;

SET hive.exec.dynamic.partition=true;  
SET hive.exec.dynamic.partition.mode=nonstrict;

INSERT OVERWRITE TABLE `posts_sample`
PARTITION (`year`, `month`)
SELECT
  `id`,
  `post_type_id`,
  `date`,
  `owner_user_id`,
  `parent_id`,
  `score`,
  `favorite_count`,
  split(regexp_replace(`tags`, '(&lt\;|&gt\;$)', ''), '&gt\;') AS `tags`,
  regexp_extract(`date`, '^(\\d{4})', 1) AS `year`,
  regexp_extract(`date`, '^(\\d{4}-\\d{2})', 1) AS `month`
FROM `posts_sample_external`
WHERE
  `id` IS NOT NULL
  AND `date` IS NOT NULL
;


Appending to query.hql


In [4]:
! hive -f query.hql 2> out.log

In [5]:
%%bash

cat out.log >&2
cat out.log | grep "Partition mskorokhod.posts_sample" | head -3 | tail -1 | sed -nr "s/.*year=([0-9]{4}).*month=([0-9]{4}\-[0-9]{2}).*numRows=([0-9]+).*/\1\t\2\t\3/p"


2008	2008-10	678



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.481 seconds
OK
Time taken: 0.017 seconds
OK
Time taken: 0.971 seconds
OK
Time taken: 0.344 seconds
OK
Time taken: 1.558 seconds
OK
Time taken: 0.152 seconds
Query ID = jovyan_20171102124545_26132a1d-0fca-4ac2-a126-6ab344cd7a9a
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1509614684555_0036, Tracking URL = http://c4bf22e8a56f:8088/proxy/application_1509614684555_0036/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1509614684555_0036
Hadoop job information for Stage-1: number of mappers: 2; number of reducers: 0
2017-11-02 12:46:06,183 Stage-1 map = 0%,  reduce = 0%
2017-11-02 12:46:24,567 Stage-1 map = 9%,  reduce = 0%, Cumulative CPU 29.54 sec
2017-11-02 12:46:30,850 Stage-1 map = 18%,  reduce = 0%, Cumulative CPU 41.93 sec
2017-11-02 12:46:37,137 St